original Polars TPC-H code: https://github.com/pola-rs/tpch

In [ ]:
import uuid
import polars as pl

from ibis_bench.utils.monitor import monitor_it
from ibis_bench.utils.read_data import get_polars_tables

from ibis_bench.queries.polars import *  # noqa

In [ ]:
# YOLO
import warnings

warnings.filterwarnings("ignore")

In [ ]:
sf = 1
n_partitions = 1
lazy = True
streaming = False

customer, lineitem, nation, orders, part, partsupp, region, supplier = (
    get_polars_tables(sf=sf, n_partitions=n_partitions, lazy=lazy)
)

In [ ]:
monitor_it(
    q1,
    sf=sf,
    n_partitions=n_partitions,
    query_number=1,
    system="polars" + "-lazy"
    if lazy
    else "-eager" + "-streaming"
    if streaming
    else "-batch",
    session_id=str(uuid.uuid4()),
    lineitem=lineitem,
)

In [ ]:
monitor_it(
    q2,
    sf=sf,
    n_partitions=n_partitions,
    query_number=2,
    system="polars" + "-lazy"
    if lazy
    else "-eager" + "-streaming"
    if streaming
    else "-batch",
    session_id=str(uuid.uuid4()),
    nation=nation,
    part=part,
    partsupp=partsupp,
    region=region,
    supplier=supplier,
)